In [1]:
import sys
sys.path.append('/opt/anaconda3/envs/pytorch/lib/python3.6')
sys.path.append('/opt/anaconda3/envs/pytorch/lib/python3.6/site-packages')
sys.path.append('/opt/anaconda3/envs/pytorch/lib/python3.6/site-packages/appv1')

In [2]:
import random
import time
import numpy as np
from tqdm import tqdm
import torch 
from torch import nn
import torch.optim as optim
import torchtext

In [3]:
from utils.dataloader import get_chABSA_DataLoaders_and_TEXT
from utils.bert import BertTokenizer

In [4]:
# JUMANの動作確認
from pyknp import Juman
juman = Juman()
result = juman.analysis("今夜の月は綺麗ですね。")
for mrph in result.mrph_list(): # 各形態素にアクセス
    print("見出し:%s, 読み:%s, 原形:%s, 品詞:%s, 品詞細分類:%s, 活用型:%s, 活用形:%s, 意味情報:%s, 代表表記:%s" \
            % (mrph.midasi, mrph.yomi, mrph.genkei, mrph.hinsi, mrph.bunrui, mrph.katuyou1, mrph.katuyou2, mrph.imis, mrph.repname))

見出し:今夜, 読み:こんや, 原形:今夜, 品詞:名詞, 品詞細分類:時相名詞, 活用型:*, 活用形:*, 意味情報:代表表記:今夜/こんや カテゴリ:時間, 代表表記:今夜/こんや
見出し:の, 読み:の, 原形:の, 品詞:助詞, 品詞細分類:接続助詞, 活用型:*, 活用形:*, 意味情報:NIL, 代表表記:
見出し:月, 読み:げつ, 原形:月, 品詞:名詞, 品詞細分類:時相名詞, 活用型:*, 活用形:*, 意味情報:代表表記:月/げつ 漢字読み:音 カテゴリ:時間, 代表表記:月/げつ
見出し:は, 読み:は, 原形:は, 品詞:助詞, 品詞細分類:副助詞, 活用型:*, 活用形:*, 意味情報:NIL, 代表表記:
見出し:綺麗です, 読み:きれいです, 原形:綺麗だ, 品詞:形容詞, 品詞細分類:*, 活用型:ナ形容詞, 活用形:デス列基本形, 意味情報:代表表記:綺麗だ/きれいだ, 代表表記:綺麗だ/きれいだ
見出し:ね, 読み:ね, 原形:ね, 品詞:助詞, 品詞細分類:終助詞, 活用型:*, 活用形:*, 意味情報:NIL, 代表表記:
見出し:。, 読み:。, 原形:。, 品詞:特殊, 品詞細分類:句点, 活用型:*, 活用形:*, 意味情報:NIL, 代表表記:


In [5]:
from config import *
from predict import predict, create_vocab_text, build_bert_model
from IPython.display import HTML, display

In [6]:
TEXT = create_vocab_text()

['その', '一方', 'で', '、', '中国', '経済', 'の', '景気', '減速', 'や', '米国', '新', '政権', 'の', '政策', '運営', '、', '英国', 'の', 'ＥＵ', '離脱', '等', 'の', 'リスク', 'に', 'より', '、', '[UNK]', 'は', '依然と', 'して', '[UNK]', '状況', 'に', 'あり', 'ます']
['その', '一方', 'で', '、', '中国', '経済', 'の', '景気', '減速', 'や', '米国', '新', '政権', 'の', '政策', '運営', '、', '英国', 'の', 'ＥＵ', '離脱', '等', 'の', 'リスク', 'に', 'より', '、', '[UNK]', 'は', '依然と', 'して', '[UNK]', '状況', 'に', 'あり', 'ます']


In [10]:
from utils.bert import get_config, BertModel,BertForchABSA, set_learned_params

from utils.bert import get_config, BertModel,BertForchABSA, set_learned_params

# The setting parameter has been saved in config.json
config = get_config(file_path="/opt/anaconda3/envs/pytorch/lib/python3.6/site-packages/appv1/weights/bert_config.json")

# build a BERT model
net_bert = BertModel(config)

# Trained BERT model params setting
net_bert = set_learned_params(
    net_bert, weights_path="/opt/anaconda3/envs/pytorch/lib/python3.6/site-packages/appv1/weights/pytorch_model.bin")




bert.embeddings.word_embeddings.weight→embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight→embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight→embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight→embeddings.LayerNorm.gamma
bert.embeddings.LayerNorm.bias→embeddings.LayerNorm.beta
bert.encoder.layer.0.attention.self.query.weight→encoder.layer.0.attention.selfattn.query.weight
bert.encoder.layer.0.attention.self.query.bias→encoder.layer.0.attention.selfattn.query.bias
bert.encoder.layer.0.attention.self.key.weight→encoder.layer.0.attention.selfattn.key.weight
bert.encoder.layer.0.attention.self.key.bias→encoder.layer.0.attention.selfattn.key.bias
bert.encoder.layer.0.attention.self.value.weight→encoder.layer.0.attention.selfattn.value.weight
bert.encoder.layer.0.attention.self.value.bias→encoder.layer.0.attention.selfattn.value.bias
bert.encoder.layer.0.attention.output.dense.weight→encoder.layer.0.attention.outpu

In [12]:
# load the finetuning module
net_trained = BertForchABSA(net_bert)
save_path = '/opt/anaconda3/envs/pytorch/lib/python3.6/site-packages/appv1/weights/bert_fine_tuning_chABSA.pth'   

net_trained.load_state_dict(torch.load(save_path, map_location='cpu'))
net_trained.eval()

BertForchABSA(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32006, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (selfattn): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertInte

In [25]:
from utils.predict import predict2

input_text = "当社の業績は多岐にわたる変動要因の影響を受ける可能性があります。有価証券報告書に記載した事業の状況、経理の状況等に関する事項のうち、投資者の判断に重要な影響を及ぼす可能性のある事項には、以下のようなものがあります。なお、文中における将来に関する事項は、当事業年度末現在において当社が判断したものであります。（１）主要市場での需要の急激な変動について当社は、主に半導体業界及びＦＰＤ業界を対象として、その生産ラインで用いられる各種生産設備部品の製造・販売を行っていますが、半導体業界におきましてシリコンサイクル、ＦＰＤ業界におきましてクリスタルサイクルと呼ばれる業界特有の好不況の波が存在します。当社におきましては、メーカーの設備投資動向に左右されない消耗品などの安定的な販売が見込める分野の受注に注力するなどの対策を行い、業績への影響を最小限にすべく努力しております。しかしながら、これらの景気変動によって、当社の業績及び財務状況に影響を与える可能性があります。"
#net_trained = build_bert_model()
net_trained.eval()
html_output = predict(input_text, net_trained)
print("======================推論結果の表示======================")
print(input_text)
display(HTML(html_output))

pred_output = predict2(input_text, net_trained)
print(pred_output)

['当社', 'の', '業績', 'は', '多岐に', 'わたる', '変動', '要因', 'の', '影響', 'を', '受ける', '可能', '性', 'が', 'あり', 'ます', '。', '有価', '証券', '報告', '書', 'に', '記載', 'した', '事業', 'の', '状況', '、', '経理', 'の', '状況', '等', 'に', '関する', '事項', 'の', 'うち', '、', '投資', '者', 'の', '判断', 'に', '重要な', '影響', 'を', '及', 'ぼ', 'す', '可', '能', '性', 'の', 'あ', 'る', '事', '項', 'に', 'は', '、', '以下', 'の', 'ような', 'もの', 'が', 'あり', 'ます', '。', 'なお', '、', '[UNK]', 'に', 'おける', '将来', 'に', '関する', '事項', 'は', '、', '当', '事業', '年度', '末', '現在', 'に', 'おいて', '当社', 'が', '判断', 'した', 'もの', 'であり', 'ます', '。', '（', '[UNK]', '）', '主要', '市場', 'で', 'の', '需要', 'の', '急激な', '変動', 'に', 'ついて', '当社', 'は', '、', '主に', '半導体', '業界', '及び', '[UNK]', '業界', 'を', '対象', 'と', 'して', '、', 'その', '生産', 'ライン', 'で', '用い', 'られる', '各種', '生産', '設備', '部品', 'の', '製造', '・', '販売', 'を', '行って', 'い', 'ます', 'が', '、', '半導体', '業界', 'に', 'おき', 'まして', 'シリコン', 'サイクル', '、', '[UNK]', '業界', 'に', 'おき', 'まして', 'クリスタル', 'サイクル', 'と', '呼ば', 'れる', '業界', '特有の', '好', '不況', 'の', '波', 'が', '存在', 'し', 'ます', '。', '当社', '

['当社', 'の', '業績', 'は', '多岐に', 'わたる', '変動', '要因', 'の', '影響', 'を', '受ける', '可能', '性', 'が', 'あり', 'ます', '。', '有価', '証券', '報告', '書', 'に', '記載', 'した', '事業', 'の', '状況', '、', '経理', 'の', '状況', '等', 'に', '関する', '事項', 'の', 'うち', '、', '投資', '者', 'の', '判断', 'に', '重要な', '影響', 'を', '及', 'ぼ', 'す', '可', '能', '性', 'の', 'あ', 'る', '事', '項', 'に', 'は', '、', '以下', 'の', 'ような', 'もの', 'が', 'あり', 'ます', '。', 'なお', '、', '[UNK]', 'に', 'おける', '将来', 'に', '関する', '事項', 'は', '、', '当', '事業', '年度', '末', '現在', 'に', 'おいて', '当社', 'が', '判断', 'した', 'もの', 'であり', 'ます', '。', '（', '[UNK]', '）', '主要', '市場', 'で', 'の', '需要', 'の', '急激な', '変動', 'に', 'ついて', '当社', 'は', '、', '主に', '半導体', '業界', '及び', '[UNK]', '業界', 'を', '対象', 'と', 'して', '、', 'その', '生産', 'ライン', 'で', '用い', 'られる', '各種', '生産', '設備', '部品', 'の', '製造', '・', '販売', 'を', '行って', 'い', 'ます', 'が', '、', '半導体', '業界', 'に', 'おき', 'まして', 'シリコン', 'サイクル', '、', '[UNK]', '業界', 'に', 'おき', 'まして', 'クリスタル', 'サイクル', 'と', '呼ば', 'れる', '業界', '特有の', '好', '不況', 'の', '波', 'が', '存在', 'し', 'ます', '。', '当社', '

In [24]:

from utils.predict import predict2

df = pd.read_csv("/Users/chenying/testsentiment.csv", encoding="utf-8-sig")
df["PREDICT"] = np.nan   #予測列を追加

net_trained.eval()  #推論モードに。

for index, row in df.iterrows():
    df.at[index, "PREDICT"] = predict2(row['text'], net_trained.numpy()[0]  # GPU環境の場合は「.cpu().numpy()」としてください。

#print(df)    
df.to_csv("/Users/chenying/predicted_test .csv", encoding="utf-8-sig", index=False)

['当社', 'の', '業績', 'は', '多岐に', 'わたる', '変動', '要因', 'の', '影響', 'を', '受ける', '可能', '性', 'が', 'あり', 'ます', '。', '有価', '証券', '報告', '書', 'に', '記載', 'した', '事業', 'の', '状況', '、', '経理', 'の', '状況', '等', 'に', '関する', '事項', 'の', 'うち', '、', '投資', '者', 'の', '判断', 'に', '重要な', '影響', 'を', '及', 'ぼ', 'す', '可', '能', '性', 'の', 'あ', 'る', '事', '項', 'に', 'は', '、', '以下', 'の', 'ような', 'もの', 'が', 'あり', 'ます', '。', 'なお', '、', '[UNK]', 'に', 'おける', '将来', 'に', '関する', '事項', 'は', '、', '当', '事業', '年度', '末', '現在', 'に', 'おいて', '当社', 'が', '判断', 'した', 'もの', 'であり', 'ます', '。', '（', '[UNK]', '）', '主要', '市場', 'で', 'の', '需要', 'の', '急激な', '変動', 'に', 'ついて', '当社', 'は', '、', '主に', '半導体', '業界', '及び', '[UNK]', '業界', 'を', '対象', 'と', 'して', '、', 'その', '生産', 'ライン', 'で', '用い', 'られる', '各種', '生産', '設備', '部品', 'の', '製造', '・', '販売', 'を', '行って', 'い', 'ます', 'が', '、', '半導体', '業界', 'に', 'おき', 'まして', 'シリコン', 'サイクル', '、', '[UNK]', '業界', 'に', 'おき', 'まして', 'クリスタル', 'サイクル', 'と', '呼ば', 'れる', '業界', '特有の', '好', '不況', 'の', '波', 'が', '存在', 'し', 'ます', '。', '当社', '